## Packages Installation

First, install the `holisticai` package if you haven't already:
```bash
!pip install holisticai[all]
```
Then, import the necessary libraries.

In [1]:
import warnings

import pandas as pd
from holisticai.bias.metrics import classification_bias_metrics
from holisticai.datasets import load_dataset
from holisticai.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

## Dataset loading

In [2]:
dataset = load_dataset('adult', protected_attribute='sex')
train_test = dataset.train_test_split(test_size=0.2, random_state=42)

train = train_test['train']
test = train_test['test']

dataset

Dataset({
        features: ['X', 'y', 'group_a', 'group_b'],
        num_rows: 45222
    })

Now that we have a clean dataset we can start defining the training and testing sets.

# 1. Adversarial Debiasing

### Traditional Implementation

In [3]:
# Define inprocessing model
from holisticai.bias.mitigation import AdversarialDebiasing

mitigator = AdversarialDebiasing(features_dim=train['X'].shape[1], batch_size=32, hidden_size=64, 
                                 adversary_loss_weight=0.1, verbose=1, use_debias=True, seed=42).transform_estimator()

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

[elapsed time: 00:00:48 | epochs: 10.00 | classifier loss:0.3288 | adversarial loss:0.6105]


,Value,Reference
Metric,,
Statistical Parity,0.099106,0
Disparate Impact,1.798032,1
Four Fifths Rule,0.556164,1
Cohen D,0.253752,0
2SD Rule,11.208130,0
Equality of Opportunity Difference,-0.100480,0
False Positive Rate Difference,0.016182,0
Average Odds Difference,-0.042149,0
Accuracy Difference,-0.094345,0


### Pipeline Implementation

In [4]:
mitigator = AdversarialDebiasing(features_dim=train['X'].shape[1], batch_size=32, hidden_size=64, adversary_loss_weight=0.1,
                                          verbose=1, use_debias=True, seed=42).transform_estimator()

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

[elapsed time: 00:01:00 | epochs: 10.00 | classifier loss:0.3304 | adversarial loss:0.6104]


,Value,Reference
Metric,,
Statistical Parity,0.098825,0
Disparate Impact,1.802405,1
Four Fifths Rule,0.554814,1
Cohen D,0.253650,0
2SD Rule,11.203698,0
Equality of Opportunity Difference,-0.105169,0
False Positive Rate Difference,0.019390,0
Average Odds Difference,-0.042890,0
Accuracy Difference,-0.099230,0


### Comparison

In [5]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional   Pipeline          
                                         Value      Value Reference
Metric                                                             
Statistical Parity                    0.099106   0.098825         0
Disparate Impact                      1.798032   1.802405         1
Four Fifths Rule                      0.556164   0.554814         1
Cohen D                               0.253752   0.253650         0
2SD Rule                             11.208130  11.203698         0
Equality of Opportunity Difference   -0.100480  -0.105169         0
False Positive Rate Difference        0.016182   0.019390         0
Average Odds Difference              -0.042149  -0.042890         0
Accuracy Difference                  -0.094345  -0.099230         0

# 2. Exponentiated Gradient

### Traditional Implementation

In [6]:
# Define inprocessing model
from holisticai.bias.mitigation import ExponentiatedGradientReduction

model = LogisticRegression()
mitigator = ExponentiatedGradientReduction(constraints="DemographicParity").transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,-0.003104,0
Disparate Impact,0.980531,1
Four Fifths Rule,0.980531,1
Cohen D,-0.008524,0
2SD Rule,-0.379158,0
Equality of Opportunity Difference,-0.305131,0
False Positive Rate Difference,-0.047765,0
Average Odds Difference,-0.176448,0
Accuracy Difference,-0.095303,0


### Pipeline Implementation

In [7]:
mitigator = ExponentiatedGradientReduction(constraints="DemographicParity").transform_estimator(model)

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

,Value,Reference
Metric,,
Statistical Parity,-0.003104,0
Disparate Impact,0.980531,1
Four Fifths Rule,0.980531,1
Cohen D,-0.008524,0
2SD Rule,-0.379158,0
Equality of Opportunity Difference,-0.305131,0
False Positive Rate Difference,-0.047765,0
Average Odds Difference,-0.176448,0
Accuracy Difference,-0.095303,0


### Comparison

In [8]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                   -0.003104 -0.003104         0
Disparate Impact                      0.980531  0.980531         1
Four Fifths Rule                      0.980531  0.980531         1
Cohen D                              -0.008524 -0.008524         0
2SD Rule                             -0.379158 -0.379158         0
Equality of Opportunity Difference   -0.305131 -0.305131         0
False Positive Rate Difference       -0.047765 -0.047765         0
Average Odds Difference              -0.176448 -0.176448         0
Accuracy Difference                  -0.095303 -0.095303         0

# 3. Grid Search Reduction

### Traditional Implementation

In [9]:
# Define inprocessing model
from holisticai.bias.mitigation import GridSearchReduction

model = LogisticRegression()
mitigator = GridSearchReduction(constraints="DemographicParity", loss='Square', min_val=-0.1, max_val=1.3, 
                                         grid_size=20).transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,0.176669,0
Disparate Impact,3.274902,1
Four Fifths Rule,0.305353,1
Cohen D,0.453875,0
2SD Rule,19.748284,0
Equality of Opportunity Difference,0.100390,0
False Positive Rate Difference,0.074568,0
Average Odds Difference,0.087479,0
Accuracy Difference,-0.110163,0


### Pipeline Implementation

In [10]:
mitigator = GridSearchReduction(constraints="DemographicParity", loss='Square', min_val=-0.1, max_val=1.3, 
                                         grid_size=20).transform_estimator(model)
# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

,Value,Reference
Metric,,
Statistical Parity,0.176669,0
Disparate Impact,3.274902,1
Four Fifths Rule,0.305353,1
Cohen D,0.453875,0
2SD Rule,19.748284,0
Equality of Opportunity Difference,0.100390,0
False Positive Rate Difference,0.074568,0
Average Odds Difference,0.087479,0
Accuracy Difference,-0.110163,0


### Comparison

In [11]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional   Pipeline          
                                         Value      Value Reference
Metric                                                             
Statistical Parity                    0.176669   0.176669         0
Disparate Impact                      3.274902   3.274902         1
Four Fifths Rule                      0.305353   0.305353         1
Cohen D                               0.453875   0.453875         0
2SD Rule                             19.748284  19.748284         0
Equality of Opportunity Difference    0.100390   0.100390         0
False Positive Rate Difference        0.074568   0.074568         0
Average Odds Difference               0.087479   0.087479         0
Accuracy Difference                  -0.110163  -0.110163         0

# 4. Meta Fair Classifier

### Traditional Implementation

In [12]:
# Define inprocessing model
from holisticai.bias.mitigation import MetaFairClassifier

model = LogisticRegression()
mitigator = MetaFairClassifier(constraint="StatisticalRate", verbose=1, seed=42).transform_estimator(_)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

[elapsed time: 00:00:04 | iter:8/8 | accuracy:0.6313 | gamma:0.6322]


,Value,Reference
Metric,,
Statistical Parity,0.085380,0
Disparate Impact,1.159979,1
Four Fifths Rule,0.862085,1
Cohen D,0.174269,0
2SD Rule,7.725735,0
Equality of Opportunity Difference,-0.002836,0
False Positive Rate Difference,-0.003188,0
Average Odds Difference,-0.003012,0
Accuracy Difference,0.082562,0


### Pipeline Implementation

In [13]:
mitigator = MetaFairClassifier(constraint="StatisticalRate", verbose=1, seed=42).transform_estimator(_)
# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

[elapsed time: 00:00:03 | iter:8/8 | accuracy:0.6313 | gamma:0.6322]


,Value,Reference
Metric,,
Statistical Parity,0.085380,0
Disparate Impact,1.159979,1
Four Fifths Rule,0.862085,1
Cohen D,0.174269,0
2SD Rule,7.725735,0
Equality of Opportunity Difference,-0.002836,0
False Positive Rate Difference,-0.003188,0
Average Odds Difference,-0.003012,0
Accuracy Difference,0.082562,0


### Comparison

In [14]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                    0.085380  0.085380         0
Disparate Impact                      1.159979  1.159979         1
Four Fifths Rule                      0.862085  0.862085         1
Cohen D                               0.174269  0.174269         0
2SD Rule                              7.725735  7.725735         0
Equality of Opportunity Difference   -0.002836 -0.002836         0
False Positive Rate Difference       -0.003188 -0.003188         0
Average Odds Difference              -0.003012 -0.003012         0
Accuracy Difference                   0.082562  0.082562         0

# 5. Prejudice Remover

### Traditional Implementation

In [15]:
# Define inprocessing model
from holisticai.bias.mitigation import PrejudiceRemover

model = LogisticRegression()
mitigator = PrejudiceRemover(maxiter=100, fit_intercept=True, verbose=1, print_interval=1).transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test, test['group_a'], test['group_b'])

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

[elapsed time: 00:00:01 | iter:5/100 | loss:13103.7994]
[elapsed time: 00:00:04 | Best Loss : 13103.7994]


,Value,Reference
Metric,,
Statistical Parity,0.108639,0
Disparate Impact,2.114219,1
Four Fifths Rule,0.472988,1
Cohen D,0.291181,0
2SD Rule,12.833175,0
Equality of Opportunity Difference,-0.037908,0
False Positive Rate Difference,0.030534,0
Average Odds Difference,-0.003687,0
Accuracy Difference,-0.110517,0


### Pipeline Implementation

In [16]:
mitigator = PrejudiceRemover(maxiter=100, fit_intercept=True, verbose=1, print_interval=1).transform_estimator(model)

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

[elapsed time: 00:00:01 | iter:5/100 | loss:13103.7994]
[elapsed time: 00:00:04 | Best Loss : 13103.7994]


,Value,Reference
Metric,,
Statistical Parity,0.108639,0
Disparate Impact,2.114219,1
Four Fifths Rule,0.472988,1
Cohen D,0.291181,0
2SD Rule,12.833175,0
Equality of Opportunity Difference,-0.037908,0
False Positive Rate Difference,0.030534,0
Average Odds Difference,-0.003687,0
Accuracy Difference,-0.110517,0


### Comparison

In [17]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional   Pipeline          
                                         Value      Value Reference
Metric                                                             
Statistical Parity                    0.108639   0.108639         0
Disparate Impact                      2.114219   2.114219         1
Four Fifths Rule                      0.472988   0.472988         1
Cohen D                               0.291181   0.291181         0
2SD Rule                             12.833175  12.833175         0
Equality of Opportunity Difference   -0.037908  -0.037908         0
False Positive Rate Difference        0.030534   0.030534         0
Average Odds Difference              -0.003687  -0.003687         0
Accuracy Difference                  -0.110517  -0.110517         0